# Week 5:  Battle of Neighborhood

### Opening a new BAKERY shop in Coimbatore, India




# 1. Import  Libraries 

In [38]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# 2. Scrap data from Wikipedia page into a DataFrame

In [39]:
# Get request and parse data
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Coimbatore').text
soup = BeautifulSoup(data, 'html.parser')

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

# create a list to store neighborhood data
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    cbe_df = pd.DataFrame({"Neighborhood": neighborhoodList})

cbe_df.head()

,Neighborhood
0,Ashokapuram
1,Avarampalayam
2,Chinniampalayam
3,"Ganapathy, Coimbatore"
4,"Gandhipuram, Coimbatore"


In [40]:
# Total 36 neighborhoods
cbe_df.shape

(36, 1)

# 3. Get the geographical coordinates

In [45]:
import geocoder # import geocoder

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Coimbatore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in cbe_df["Neighborhood"].tolist() ]
coords

[[11.083420000000046, 76.97482000000008],
 [11.026670000000024, 76.98211000000003],
 [11.032350000000065, 77.07564000000008],
 [11.085660000000075, 76.92786000000007],
 [11.019260000000031, 76.96792000000005],
 [11.045280000000048, 76.94729000000007],
 [11.078090000000032, 77.03755000000007],
 [11.11516000000006, 76.99552000000006],
 [10.958480000000066, 76.95343000000008],
 [10.936190000000067, 76.97259000000008],
 [11.01311000000004, 76.89259000000004],
 [11.117590000000064, 76.95321000000007],
 [11.009760000000028, 76.99710000000005],
 [10.994160000000022, 76.96629000000007],
 [11.008250000000032, 77.05119000000008],
 [11.036200000000065, 77.00686000000007],
 [10.975620000000049, 76.91261000000003],
 [10.955050000000028, 76.99281000000008],
 [11.007550000000037, 76.99341000000004],
 [11.00735000000003, 76.95310000000006],
 [11.012860000000046, 76.96456000000006],
 [10.992510000000038, 76.99117000000007],
 [11.028110000000027, 76.96543000000008],
 [11.02493000000004, 76.9450800000000

# 4. Create a Dataframe and save as CSV

In [52]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
cbe_df['Latitude'] = df_coords['Latitude']
cbe_df['Longitude'] = df_coords['Longitude']


# save the DataFrame as CSV file
cbe_df.to_csv("cbe_df.csv", index=False)
cbe_df


,Neighborhood,Latitude,Longitude
0,Ashokapuram,11.083420,76.974820
1,Avarampalayam,11.026670,76.982110
2,Chinniampalayam,11.032350,77.075640
3,"Ganapathy, Coimbatore",11.085660,76.927860
4,"Gandhipuram, Coimbatore",11.019260,76.967920
5,Goundampalayam,11.045280,76.947290
6,Kalapatti,11.078090,77.037550
7,Keeranatham,11.115160,76.995520
8,Kuniyamuthur,10.958480,76.953430
9,Kurichi,10.936190,76.972590


# 5 Create a map of Coimbatore with neighborhoods superimposed on top

In [59]:

# get the coordinates of Kuala Lumpur
address = 'Coimbatore'

geolocator = Nominatim(user_agent="my city")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Coimbatore,India {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Coimbatore,India 11.0018115, 76.9628425.


In [62]:
# create map of Coimbatore using latitude and longitude values
map_cbe = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(cbe_df['Latitude'], cbe_df['Longitude'], cbe_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_cbe)  
# save the map as HTML file
map_cbe.save('map_kl.html')  
map_cbe


# 6 Use the Foursquare API to explore the neighborhoods

In [63]:
# define Foursquare Credentials and Version
CLIENT_ID = 'XWCOA4HGPQNJLVWOEVJSWPNIFZJZQJF2NWN0PGYKOPNFXE2Z' # your Foursquare ID
CLIENT_SECRET = 'LDO0RSKTXONWPDBMRTMKWYKKSQLR4XZTVUXFDIIF5GX0VRFZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your Foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your Foursquare credentails:
CLIENT_ID: XWCOA4HGPQNJLVWOEVJSWPNIFZJZQJF2NWN0PGYKOPNFXE2Z
CLIENT_SECRET:LDO0RSKTXONWPDBMRTMKWYKKSQLR4XZTVUXFDIIF5GX0VRFZ


**Get the top 100 venues that are within a radius of 2000 meters**

In [64]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(cbe_df['Latitude'], cbe_df['Longitude'], cbe_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [66]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(871, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashokapuram,11.08342,76.97482,Sri Raghu Restaurant,11.081270,76.983801,Indian Restaurant
1,Ashokapuram,11.08342,76.97482,Punjabi Food,11.082150,76.986912,Indian Restaurant
2,Ashokapuram,11.08342,76.97482,Walnut,11.082174,76.988164,Ice Cream Shop
3,Ashokapuram,11.08342,76.97482,Punjabi Dhabba,11.080804,76.988554,Indian Restaurant
4,Ashokapuram,11.08342,76.97482,Jay Vees Bakery,11.081502,76.989291,Bakery


In [68]:
## Let's check how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashokapuram,7,7,7,7,7,7
Avarampalayam,30,30,30,30,30,30
Chinniampalayam,1,1,1,1,1,1
"Ganapathy, Coimbatore",5,5,5,5,5,5
"Gandhipuram, Coimbatore",63,63,63,63,63,63
Goundampalayam,5,5,5,5,5,5
Kalapatti,4,4,4,4,4,4
Keeranatham,5,5,5,5,5,5
Kuniyamuthur,4,4,4,4,4,4


In [69]:
##Let's find out unique categories from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 75 uniques categories.


In [70]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Ice Cream Shop', 'Bakery', 'Arcade',
       'Vegetarian / Vegan Restaurant', 'Hotel', 'Fried Chicken Joint',
       'Chinese Restaurant', 'Coffee Shop', 'Nightclub', 'Motel',
       'Snack Place', 'Asian Restaurant', 'Restaurant',
       'South Indian Restaurant', 'Bus Station', 'Farmers Market',
       'Train Station', 'Indie Movie Theater', 'Dessert Shop',
       'Multiplex', 'Shopping Mall', 'Café', 'Clothing Store',
       'Fast Food Restaurant', 'Park', 'Department Store', 'Food Court',
       "Women's Store", 'Miscellaneous Shop', 'Stadium', 'Board Shop',
       'Jewelry Store', 'Bistro', 'Food Truck', 'Theme Restaurant',
       'Business Service', 'Athletics & Sports', 'ATM', 'Diner',
       'Indian Sweet Shop', 'Italian Restaurant', 'Hostel',
       'Burger Joint', 'Pizza Place', 'Gym', 'BBQ Joint', 'Steakhouse',
       'Bar', 'Sandwich Place'], dtype=object)

# 7 Analyze Each Neighborhood

### Label encoding of venue categories

In [92]:
# one hot encoding
cbe_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cbe_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column 
fixed_columns = [cbe_onehot.columns[-1]] + list(cbe_onehot.columns[:-1])
cbe_onehot = cbe_onehot[fixed_columns]

print(cbe_onehot.shape)
cbe_onehot.head()



(871, 76)


,Neighborhood,ATM,Accessories Store,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Bistro,Board Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gym,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Kids Store,Market,Miscellaneous Shop,Motel,Motorcycle Shop,Movie Theater,Multiplex,Nightclub,North Indian Restaurant,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Restaurant,Sandwich Place,Shop & Service,Shopping Mall,Snack Place,South Indian Restaurant,Sports Club,Stadium,Steakhouse,Tea Room,Tennis Stadium,Theme Restaurant,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Voting Booth,Warehouse Store,Women's Store
0,Ashokapuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashokapuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashokapuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashokapuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashokapuram,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Mean of venues for each neighor

In [93]:
## group rows by neighborhood and take mean venue category
cbe_grouped = cbe_onehot.groupby(["Neighborhood"]).mean().reset_index()
print(cbe_grouped.shape)
cbe_grouped.head()

(36, 76)


,Neighborhood,ATM,Accessories Store,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Bistro,Board Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Department Store,Dessert Shop,Diner,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gym,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Kids Store,Market,Miscellaneous Shop,Motel,Motorcycle Shop,Movie Theater,Multiplex,Nightclub,North Indian Restaurant,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Restaurant,Sandwich Place,Shop & Service,Shopping Mall,Snack Place,South Indian Restaurant,Sports Club,Stadium,Steakhouse,Tea Room,Tennis Stadium,Theme Restaurant,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Voting Booth,Warehouse Store,Women's Store
0,Ashokapuram,0.0,0.0,0.142857,0.000000,0.0,0.0,0.285714,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.142857,0.0,0.428571,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,Avarampalayam,0.0,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.066667,0.066667,0.0,0.366667,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.066667,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.000000
2,Chinniampalayam,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000
3,"Ganapathy, Coimbatore",0.0,0.0,0.000000,0.000000,0.0,0.0,0.200000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.400000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.200000,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,"Gandhipuram, Coimbatore",0.0,0.0,0.000000,0.111111,0.0,0.0,0.015873,0.0,0.0,0.015873,0.015873,0.0,0.0,0.015873,0.0,0.0,0.047619,0.015873,0.095238,0.000000,0.015873,0.031746,0.0,0.0,0.000000,0.015873,0.015873,0.0,0.0,0.000000,0.0,0.0,0.0,0.063492,0.031746,0.0,0.190476,0.0,0.015873,0.0,0.015873,0.0,0.0,0.0,0.015873,0.000000,0.0,0.0,0.079365,0.015873,0.0,0.015873,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.031746,0.015873,0.000000,0.0,0.015873,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.015873


## New Dataframe for Bakery

In [139]:
len(cbe_grouped[cbe_grouped["Bakery"] > 0])


19

In [140]:
cbe_ds = cbe_grouped[["Neighborhood","Bakery"]]
cbe_ds.head()

,Neighborhood,Bakery
0,Ashokapuram,0.285714
1,Avarampalayam,0.000000
2,Chinniampalayam,0.000000
3,"Ganapathy, Coimbatore",0.200000
4,"Gandhipuram, Coimbatore",0.015873


# 8.Cluster Neighborhoods
**Run k-means to cluster the neighborhoods of Coimbatore into 3 clusters**

In [141]:
# set number of clusters
kclusters = 3
cbe_clustering = cbe_ds.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cbe_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 2, 1, 2, 1, 2, 0, 1])

In [142]:

# create a new dataframe that includes the cluster as well 
cbe_merged = cbe_ds.copy()

# add clustering labels
cbe_merged["Cluster Labels"] = kmeans.labels_
cbe_merged.rename(columns={"Neighborhood": "Neighborhood"}, inplace=True)
cbe_merged.head()



,Neighborhood,Bakery,Cluster Labels
0,Ashokapuram,0.285714,2
1,Avarampalayam,0.000000,1
2,Chinniampalayam,0.000000,1
3,"Ganapathy, Coimbatore",0.200000,2
4,"Gandhipuram, Coimbatore",0.015873,1


In [143]:
# merge dataframes to add latitude/longitude for each neighborhood
cbe_merged = cbe_merged.join(cbe_df.set_index("Neighborhood"), on="Neighborhood")
## Sort clusters 
cbe_merged.sort_values(["Cluster Labels"], inplace=True)
cbe_merged

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
8,Kuniyamuthur,0.500000,0,10.958480,76.953430
17,Podanur,0.000000,1,10.955050,76.992810
32,Veerakeralam,0.000000,1,11.006570,76.912690
31,Vadavalli,0.000000,1,11.024220,76.892950
30,Ukkadam,0.027778,1,10.989680,76.964100
29,Udayampalayam,0.000000,1,11.010120,76.995910
27,Tatabad,0.033898,1,11.024690,76.957120
26,Sukrawarpettai,0.023256,1,11.001493,76.962541
25,Sivananda Colony,0.033898,1,11.024690,76.957120
23,Sai Baba colony,0.111111,1,11.024930,76.945080


# 9 visualize the resulting clusters

In [144]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cbe_merged['Latitude'], cbe_merged['Longitude'], cbe_merged['Neighborhood'], cbe_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [145]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# 10 OBSERVATION

**I see that Cluster 0 has very less Bakeries  compared to Cluster 1 and 2. 
Hence neighborhoods of cluster 0 has lot scope for opening new Bakeries. I will suggest opening new bakeries in and around neighborhoods of Cluster 0**